<a href="https://colab.research.google.com/github/Donglllai/Supply-chain/blob/main/HW0306%20Donglai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

# Product dictionary
products = {
    1: {"demand": 1000, "common_ordering_cost": 150, "specific_ordering_cost": 20, "unit_cost": 50, "holding_cost_rate": 0.15},
    2: {"demand": 300, "common_ordering_cost": 150, "specific_ordering_cost": 25, "unit_cost": 60, "holding_cost_rate": 0.15},
    3: {"demand": 100, "common_ordering_cost": 150, "specific_ordering_cost": 30, "unit_cost": 30, "holding_cost_rate": 0.15},
    4: {"demand": 50, "common_ordering_cost": 150, "specific_ordering_cost": 50, "unit_cost": 30, "holding_cost_rate": 0.15},
}

# EOQ cost function
def calculate_eoq_cost(demand, ordering_cost, unit_cost, holding_cost_rate):
    holding_cost = unit_cost * holding_cost_rate
    order_quantity = np.sqrt((2 * demand * ordering_cost) / holding_cost)
    num_orders = demand / order_quantity
    total_ordering_cost = num_orders * ordering_cost
    total_holding_cost = (order_quantity / 2) * holding_cost
    return total_ordering_cost + total_holding_cost, order_quantity, num_orders

# Strategy 1: Independent Sourcing
independent_cost = sum(
    calculate_eoq_cost(
        data["demand"],
        data["common_ordering_cost"] + data["specific_ordering_cost"],
        data["unit_cost"],
        data["holding_cost_rate"]
    )[0]
    for data in products.values()
)

# Strategy 2: Same Frequency Sourcing
total_demand = sum(data["demand"] for data in products.values())
total_ordering_cost = sum(data["specific_ordering_cost"] for data in products.values()) + products[1]["common_ordering_cost"]
avg_unit_cost = np.mean([data["unit_cost"] for data in products.values()])
avg_holding_cost_rate = np.mean([data["holding_cost_rate"] for data in products.values()])
same_freq_cost, same_freq_q, same_freq_orders = calculate_eoq_cost(total_demand, total_ordering_cost, avg_unit_cost, avg_holding_cost_rate)

# Strategy 3: Tailored Aggregation
eoq_values = {
    p: calculate_eoq_cost(data["demand"], data["specific_ordering_cost"], data["unit_cost"], data["holding_cost_rate"])[1]
    for p, data in products.items()
}
min_eoq = min(eoq_values.values())
freq_ratios = {p: round(eoq_values[p] / min_eoq) for p in products}

# Finding common ordering cycle time
common_cycle_time = min(
    products[p]["demand"] / (freq_ratios[p] * eoq_values[p]) for p in products
)

# Computing costs for tailored aggregation
tailored_cost = sum(
    calculate_eoq_cost(
        data["demand"],
        products[1]["common_ordering_cost"] / min(freq_ratios.values()) + data["specific_ordering_cost"],
        data["unit_cost"],
        data["holding_cost_rate"]
    )[0]
    for data in products.values()
)

# Creating results dataframe
results_df = pd.DataFrame({
    "Strategy": ["Independent Sourcing", "Same Frequency Sourcing", "Tailored Aggregation"],
    "Annual Operational Cost": [independent_cost, same_freq_cost, tailored_cost]
})

# Display results
print(results_df)



                  Strategy  Annual Operational Cost
0     Independent Sourcing              3271.475283
1  Same Frequency Sourcing              2254.786575
2     Tailored Aggregation              3271.475283
